## ETL Process for Neo CSV file
- Read Neo CSV from AWS S3
- Load Neo DataFrame into AWS RDS neo database

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-15 00:18:28--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-01-15 00:18:28 (6.50 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Neo_Csv").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Import pySpark libraries
from pyspark import SparkFiles
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [18]:
csv_filename = "sbdb_query_results_asteroid_neo.csv"
url_endpoint = f"https://ucb-neo-project.s3.us-east-2.amazonaws.com/csv/{csv_filename}"

In [19]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = url_endpoint
spark.sparkContext.addFile(url)

In [28]:
df_neo = spark.read.csv(SparkFiles.get(csv_filename), sep=",", header=True)

In [29]:
df_neo.printSchema()

root
 |-- spkid: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- pdes: string (nullable = true)
 |-- name: string (nullable = true)
 |-- prefix: string (nullable = true)
 |-- neo: string (nullable = true)
 |-- pha: string (nullable = true)
 |-- H: string (nullable = true)
 |-- G: string (nullable = true)
 |-- M1: string (nullable = true)
 |-- M2: string (nullable = true)
 |-- K1: string (nullable = true)
 |-- K2: string (nullable = true)
 |-- PC: string (nullable = true)
 |-- diameter: string (nullable = true)
 |-- extent: string (nullable = true)
 |-- albedo: string (nullable = true)
 |-- rot_per: string (nullable = true)
 |-- GM: string (nullable = true)
 |-- BV: string (nullable = true)
 |-- UB: string (nullable = true)
 |-- IR: string (nullable = true)
 |-- spec_B: string (nullable = true)
 |-- spec_T: string (nullable = true)
 |-- H_sigma: string (nullable = true)
 |-- diameter_sigma: string (nullable = true)
 |-- orbit_id: string (nullable = true)
 |-- epoc

In [30]:
df_neo = df_neo.drop('name','prefix',
'neo','G','M1','M2','K1','K2','PC',
'diameter','extent','albedo','rot_per','GM','BV',
'UB','IR','spec_B','spec_T','diameter_sigma','equinox',
'n_del_obs_used','n_dop_obs_used','two_body','A1','A2','A3','DT')

In [37]:
df_neo.printSchema()

root
 |-- spkid: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- pdes: string (nullable = true)
 |-- pha: string (nullable = true)
 |-- H: string (nullable = true)
 |-- H_sigma: string (nullable = true)
 |-- orbit_id: string (nullable = true)
 |-- epoch: string (nullable = true)
 |-- epoch.mjd: string (nullable = true)
 |-- epoch.cal: string (nullable = true)
 |-- e: string (nullable = true)
 |-- a: string (nullable = true)
 |-- q: string (nullable = true)
 |-- i: string (nullable = true)
 |-- om: string (nullable = true)
 |-- w: string (nullable = true)
 |-- ma: string (nullable = true)
 |-- ad: string (nullable = true)
 |-- n: string (nullable = true)
 |-- tp: string (nullable = true)
 |-- tp.cal: string (nullable = true)
 |-- per: string (nullable = true)
 |-- per.y: string (nullable = true)
 |-- moid: string (nullable = true)
 |-- moid.ld: string (nullable = true)
 |-- moid_jup: string (nullable = true)
 |-- t_jup: string (nullable = true)
 |-- sigma_e: strin

In [65]:
df_neo = (df_neo
  .withColumnRenamed('per.y', 'per_y')
  .withColumnRenamed('moid.ld', 'moid_ld')
  .withColumnRenamed('tp.cal', 'tp_cal')
)

In [66]:
df_neo.printSchema()

root
 |-- spkid: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- pdes: string (nullable = true)
 |-- pha: string (nullable = true)
 |-- H: string (nullable = true)
 |-- H_sigma: string (nullable = true)
 |-- orbit_id: string (nullable = true)
 |-- epoch: string (nullable = true)
 |-- epoch.mjd: string (nullable = true)
 |-- epoch.cal: string (nullable = true)
 |-- e: string (nullable = true)
 |-- a: string (nullable = true)
 |-- q: string (nullable = true)
 |-- i: string (nullable = true)
 |-- om: string (nullable = true)
 |-- w: string (nullable = true)
 |-- ma: string (nullable = true)
 |-- ad: string (nullable = true)
 |-- n: string (nullable = true)
 |-- tp: string (nullable = true)
 |-- tp_cal: string (nullable = true)
 |-- per: string (nullable = true)
 |-- per_y: string (nullable = true)
 |-- moid: string (nullable = true)
 |-- moid_ld: string (nullable = true)
 |-- moid_jup: string (nullable = true)
 |-- t_jup: string (nullable = true)
 |-- sigma_e: strin

In [69]:
final_df = ( df_neo
  .transform(lambda df: df.withColumn("h", df["h"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("h_sigma", df["h_sigma"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("epoch", df["epoch"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("e", df["e"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("a", df["a"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("q", df["q"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("i", df["i"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("om", df["om"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("w", df["w"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("ma", df["ma"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("ad", df["ad"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("n", df["n"].cast(T.DecimalType(precision=24, scale=16))))    
  .transform(lambda df: df.withColumn("tp", df["tp"].cast(T.DecimalType(precision=24, scale=16))))
  #.transform(lambda df: df.withColumn("tp_cal", df["tp_cal"].cast(T.DateType())))  
  .transform(lambda df: df.withColumn("per", df["per"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("per_y", df["per_y"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("moid", df["moid"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("moid_ld", df["moid_ld"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("moid_jup", df["moid_jup"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("t_jup", df["t_jup"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("sigma_e", df["sigma_e"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("sigma_a", df["sigma_e"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("sigma_q", df["sigma_q"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("sigma_i", df["sigma_i"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("sigma_om", df["sigma_om"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("sigma_w", df["sigma_w"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("sigma_ma", df["sigma_ma"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("sigma_ad", df["sigma_ad"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("sigma_n", df["sigma_n"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("sigma_tp", df["sigma_tp"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("sigma_per", df["sigma_per"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("data_arc", df["data_arc"].cast(T.DecimalType(precision=24, scale=16))))
  .transform(lambda df: df.withColumn("first_obs", df["first_obs"].cast(T.DateType())))
  .transform(lambda df: df.withColumn("last_obs", df["last_obs"].cast(T.DateType())))
  .transform(lambda df: df.withColumn("n_obs_used", df["n_obs_used"].cast(T.IntegerType())))
  .transform(lambda df: df.withColumn("condition_code", df["condition_code"].cast(T.IntegerType())))
  .transform(lambda df: df.withColumn("rms", df["rms"].cast(T.DecimalType(precision=24, scale=16))))
)

In [70]:
final_df.printSchema()

root
 |-- spkid: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- pdes: string (nullable = true)
 |-- pha: string (nullable = true)
 |-- h: decimal(24,16) (nullable = true)
 |-- h_sigma: decimal(24,16) (nullable = true)
 |-- orbit_id: string (nullable = true)
 |-- epoch: decimal(24,16) (nullable = true)
 |-- epoch.mjd: string (nullable = true)
 |-- epoch.cal: string (nullable = true)
 |-- e: decimal(24,16) (nullable = true)
 |-- a: decimal(24,16) (nullable = true)
 |-- q: decimal(24,16) (nullable = true)
 |-- i: decimal(24,16) (nullable = true)
 |-- om: decimal(24,16) (nullable = true)
 |-- w: decimal(24,16) (nullable = true)
 |-- ma: decimal(24,16) (nullable = true)
 |-- ad: decimal(24,16) (nullable = true)
 |-- n: decimal(24,16) (nullable = true)
 |-- tp: decimal(24,16) (nullable = true)
 |-- tp_cal: string (nullable = true)
 |-- per: decimal(24,16) (nullable = true)
 |-- per_y: decimal(24,16) (nullable = true)
 |-- moid: decimal(24,16) (nullable = true)
 |-- mo

In [71]:
final_df.show(25)

+-------+--------------------+----+---+-------------------+-------+--------+--------------------+---------+------------+------------------+------------------+------------------+-------------------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+------------+--------------------+------------------+------------------+--------------------+------------------+------------------+--------------+--------------+--------------+------------------+------------------+------------------+------------------+--------------+-------------+------------------+------------------+-----+----------+--------------------+----------+----------+----------+--------------+------------------+
|  spkid|           full_name|pdes|pha|                  h|h_sigma|orbit_id|               epoch|epoch.mjd|   epoch.cal|                 e|                 a|                 q|                  i|                  om|                   w|                 

In [72]:
# import getpass module
from getpass import getpass

In [73]:
def load_data_aws_rds(df, mode, table_name):
  """
  Load data in dataframe arg df into aws rds neo database

  args:
    df: dataframe containing source data to load into database
    mode: write mode ie. append, overwrite
    table_name: name of table in database to load data into
  """

  password = getpass('Enter database password')

  # Configure settings for RDS
  jdbc_url="jdbc:postgresql://neo-db.ctohlxwhjvlb.us-east-1.rds.amazonaws.com:5432/neo"
  config = {"user":"postgres", 
            "password": password, 
            "driver":"org.postgresql.Driver"}
  
  mode = 'overwrite'
  df.write.jdbc(url=jdbc_url, table=table_name, mode=mode, properties=config)

In [74]:
table_name = 'public.neo'
mode = 'overwrite'

In [75]:
load_data_aws_rds(final_df, mode, table_name)

Enter database password··········


In [76]:
df.groupBy('pdes').count().show(50)

+----------+-----+
|      pdes|count|
+----------+-----+
|      5645|    1|
|     35670|    1|
|    153814|    1|
|    189552|    1|
|    205378|    1|
|    231134|    1|
|    265187|    1|
|    302830|    1|
|    371660|    1|
|    417816|    1|
|    434007|    1|
|    480004|    1|
|    488474|    1|
|    498125|    1|
|    501878|    1|
|    508905|    1|
|    523803|    1|
|    526449|    1|
|    533990|    1|
| 1997 GD32|    1|
| 1999 RA32|    1|
|  2000 DL8|    1|
|  2001 UB5|    1|
|  2002 AQ2|    1|
| 2003 WH98|    1|
|   2004 GA|    1|
| 2004 GC19|    1|
|  2004 MO4|    1|
| 2005 QL76|    1|
| 2005 YW93|    1|
| 2007 TH15|    1|
|  2007 UD6|    1|
|2007 VP243|    1|
|  2008 CY4|    1|
|2008 CA119|    1|
|  2008 HA2|    1|
|   2008 JC|    1|
|  2008 TE2|    1|
|   2010 BQ|    1|
| 2010 JW39|    1|
| 2010 JL88|    1|
|   2010 KD|    1|
| 2010 TF54|    1|
| 2010 TP55|    1|
|  2011 JA8|    1|
| 2011 LK19|    1|
| 2011 OD18|    1|
| 2011 QF48|    1|
|2012 BD124|    1|
|2012 TP274|